## Ensiksi syötetään alkutiedot 

Alkutietojen muuttujat:
- bp = bruttopalkka (€/vuosi)
- ika = Veronmaksajan ikä
- k_k = kuulun kirkkoon (True/False)
- t_m = Työttömyyskassan jäsenmaksu (€/vuosi)
- mk = Matka kulut per vuosi
- mko = Matkakulujen omavastuu per vuosi
- k_kunta = Kotikunta verotusvuonna

In [32]:
bp = 40000
k_k = True
t_m = 0
mk = 0
mko = 750

## Puhdas ansiotulo

Ensiksi lasketaan verotuksessa puhdas ansiotulo (pat). Tämä saadaan vähentämällä luonnolliset vähennykset bruttopalkasta. Ansiotulosta vähennetään aina tulonhankkimisvähennys (thv)

In [33]:
thv = 750

# Lasketaan ylittääkö matkakustannukset omavastuun.
if mk < mko:
    mtv = 0
else:
    mtv = mk - mko

pat = bp - t_m - mk - thv

print('Puhdas ansiotulo on: {}€'.format(pat))

Puhdas ansiotulo on: 39250€


## Sosiaalivakuutusmaksut ja sairausvakuutusmaksu

Tämän jälkeen vähennetään työeläkemaksu

In [34]:
if ika >= 17 and ika <= 52:
    telm_p = 0.0715
elif ika >= 53 and ika <= 62:
    telm_p = 0.0865
elif ika >= 63 and 67 <= 62:
    telm_p = 0.0715
else:
    telm_p = 0

tyel_m = pat*telm_p
print('TyEL maksu: {:.1f}€'.format(tyel_m))

TyEL maksu: 2806.4€


Seuraavaksi lasketaan työttömyysvakuutusmaksu, jos työntekijä on 17 - 64 -vuotias

In [35]:
if ika >= 17 and ika <= 64:
    tvm_p = 0.014
else:
    tvm_p = 0.0

tvm = pat*tvm_p
print('Työttömyysvakuutusmaksu maksu: {:.1f}€'.format(tvm))

Työttömyysvakuutusmaksu maksu: 549.5€


Seuraavaksi lasketaan sairausvakuutuksen päivärahamaksu. Vuositulot ovat alle 14766€. Maksua ei peritä

In [36]:
if ika >= 16 and ika <= 67 and pat >= 14766:
    prm_p = 0.0136
else:
    prm_p = 0


svm = pat*prm_p
print('Työttömyysvakuutusmaksu maksu: {:.1f}€'.format(svm))

Työttömyysvakuutusmaksu maksu: 533.8€


Sitten lasketaan sairaanhoitomaksu palkkatulosta (0,68%)

In [37]:
shm = pat*0.0068
print('Työttömyysvakuutusmaksu maksu: {:.1f}€'.format(shm))

Työttömyysvakuutusmaksu maksu: 266.9€


## Verotettava tulo sosiaalivakuutusmaksujen ja sairausvakuutusmaksun jälkeen

In [38]:
v_t = pat - shm - svm - tvm - tyel_m
print('Verotettavat tulo: {:.1f}€'.format(v_t))

Verotettavat tulo: 35093.4€


## Kunnallisvero

Sitten lasketaan kunnallisverot. Ensin aloitetaan kunnallisverotuksen asiotulovähennykestä (atv). Kunnallisveron ansiotulovähennys lasketaan ureasta eri tuloista:
- Veron alaisesta palkkatulosta
- Muusta toiselle suoritetusta työstä, tehtävästä tai palveluksesta saadut ansiotulot
- Ansiotulosta pidätettävistä käyttökorvauksista
- Ansiotulona verotettavasta  osingosta
- Jaettavasta yritystulosta, joka on asiotulon-osuutta
- Uhtymän osakkaan elinkeinotoiminnan tai maatalouden ansiotulo-osuuden osuudesta

Tässä esimerkissä kuitenkin keskitytään vain veron alaiseen palkkatuloon

In [ ]:

if bp <= 2500:
    atv = 0
elif bp  > 2500 and bp  <= 7230:
    atv = (bt - 2500)*0.51
elif bp  > 7230:
    atv = (7230–2500) × 0.51 + (bp – 7230) × 0.28
